# Paralelismo (local)

### Coroutines

In [ ]:
# creamos los canales
c1 = Channel(32)
c2 = Channel(32)

In [ ]:
# escribimos unos datos en los canales
put!(c1, "Apu")
put!(c1, "Homero")
put!(c1, "Bart")
put!(c1, 80)
put!(c1, 44)
put!(c1, 55)

In [ ]:
accion(dato::Int64) = dato + 2
accion(dato::String) = "Hola " * dato * "!!"

# lee un item de c1, lo printea y lo escribe en c2
function foo()
    while true
        dato = take!(c1)
        println(dato)
        resultado = accion(dato)
        put!(c2, resultado)   
    end
end

In [ ]:
# con @async hacemos que varias instancias de foo() estén activas concurrentemente.
for i in 1:size(c1.data)[1]
    @async foo()
end

In [ ]:
for i in 1:size(c2.data)[1]
    data= take!(c2)
    println(data)
end

### Distributed processing

In [ ]:
using Distributed

In [ ]:
# primero vamos a ver cuántos procesos están corriendo
println(nprocs())

In [ ]:
# agregamos 4 workers más
addprocs(4)

In [ ]:
println(nprocs())

In [ ]:
workers()

In [ ]:
# borramos 2
rmprocs(4, 5)

In [ ]:
workers()

In [ ]:
# vamos a ver qué id tiene el proceso local
println("Hola, soy el worker " * string(myid()) * " y los quiero mucho")

In [ ]:
# vamos a ver qué ids tienen los procesos distribuidos
for w in workers()
    rref = remotecall(myid, w)
    sleep(1)
    println("Hola, soy el worker " * string(fetch(rref)) * " y no los quiero tanto")
end

### Remotecall + Fetch

In [ ]:
dim = 5
res = remotecall(rand, 2, dim, dim)
res2 = remotecall(rand, 3, dim, dim)
resultado = fetch(remotecall(*, 6, fetch(res), fetch(res2)))

### Spawn

In [ ]:
# definimos una funcion a todos los workers
@everywhere function numero_rand(veces)
    resultado::Int = 0
    for i = 1:veces
        resultado += rand(Bool)
    end
    resultado
end

In [ ]:
a = @spawn numero_rand(100)

In [ ]:
b = @spawn numero_rand(10)

In [ ]:
println(fetch(a)+fetch(b))